# LangChain 1.0 Tool Stream Writer

Ref:

- https://docs.langchain.com/oss/python/langchain/tools#stream-writer

In [0]:
%pip install -U langchain>=1.0.0 langchain_openai>=1.0.0 mlflow

%restart_python

## モデルとの接続

In [0]:
# from langchain_openai import ChatOpenAI
from langchain.chat_models import init_chat_model
import mlflow
import os

mlflow.langchain.autolog()

creds = mlflow.utils.databricks_utils.get_databricks_host_creds()
model = init_chat_model(
    "openai:databricks-gpt-oss-20b",
    api_key=creds.token,
    base_url=creds.host + "/serving-endpoints",
)

## Agent作成・実行

In [0]:
from langchain.tools import tool, ToolRuntime
from langgraph.config import get_stream_writer

@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """Get weather for a given city."""
    # writer = runtime.stream_writer
    writer = get_stream_writer()

    # Stream custom updates as the tool executes
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")

    # with mlflow.start_span(name="user_role_prompt") as span:
    #     span.set_inputs(city)
    #     span.set_outputs(f"It's always sunny in {city}!")

    return f"It's always sunny in {city}!"

In [0]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent

store = InMemoryStore()
agent = create_agent(model, tools=[get_weather], store=store)

# First session: save user info
# agent.invoke({
#     "messages": [{"role": "user", "content": "東京の天気を教えて"}]
# })

for stream_mode, chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["custom", "updates"],
):
    print(f"stream_mode: {stream_mode}")
    print(f"content: {chunk}")
    print("\n")
    # for step, data in chunk.items():
    #     print(f"step: {step}")
    #     print(f"content: {data['messages'][-1].content_blocks}")